<a href="https://colab.research.google.com/github/heyayesha/2022TTP-Application-Project/blob/main/AI_%26_ML_to_predict_data_(COVID_19).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

1. **Importing** **Libraries**

In [1]:
import numpy as np 
import pandas as pd

import plotly.express as px
import plotly.graph_objects as go #import plotly.figure_factory as ff
from plotly.subplots import make_subplots #from plotly.colors import n_colors
from sklearn.preprocessing import MinMaxScaler # Minmax scaler
import itertools

pd.set_option('display.max_columns', 5000000)
pd.set_option('display.max_rows', 50000000)

import warnings  #To remove un-necessary warnings
warnings.filterwarnings('ignore')

import xgboost as xgb

#Importing SKlearn models
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import Perceptron
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC, LinearSVC
from sklearn.model_selection import cross_val_score
from sklearn.metrics import precision_score, recall_score, accuracy_score, r2_score
from sklearn.model_selection import RandomizedSearchCV, train_test_split
from sklearn.linear_model import LinearRegression  
from sklearn.preprocessing import PolynomialFeatures
from sklearn import linear_model

from pandas.core.common import flatten

**2. Importing Dataset from kagggle**

In [4]:
confirmed_df = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv')
deaths_df = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_global.csv')
recoveries_df = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_recovered_global.csv')
latest_data = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_daily_reports/08-22-2020.csv')
us_medical_data = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_daily_reports_us/08-22-2020.csv')

**3. Pre-Processing Data**

In [5]:
confirmed_df.head()

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,1/28/20,1/29/20,1/30/20,1/31/20,2/1/20,2/2/20,2/3/20,2/4/20,2/5/20,2/6/20,2/7/20,2/8/20,2/9/20,2/10/20,2/11/20,2/12/20,2/13/20,2/14/20,2/15/20,2/16/20,2/17/20,2/18/20,2/19/20,2/20/20,2/21/20,2/22/20,2/23/20,2/24/20,2/25/20,2/26/20,2/27/20,2/28/20,2/29/20,3/1/20,3/2/20,3/3/20,3/4/20,3/5/20,3/6/20,3/7/20,3/8/20,3/9/20,3/10/20,3/11/20,3/12/20,3/13/20,3/14/20,3/15/20,3/16/20,3/17/20,3/18/20,3/19/20,3/20/20,3/21/20,3/22/20,3/23/20,3/24/20,3/25/20,3/26/20,3/27/20,3/28/20,3/29/20,3/30/20,3/31/20,4/1/20,4/2/20,4/3/20,4/4/20,4/5/20,4/6/20,4/7/20,4/8/20,4/9/20,4/10/20,4/11/20,4/12/20,4/13/20,4/14/20,4/15/20,4/16/20,4/17/20,4/18/20,4/19/20,4/20/20,4/21/20,4/22/20,4/23/20,4/24/20,4/25/20,4/26/20,4/27/20,4/28/20,4/29/20,4/30/20,5/1/20,5/2/20,5/3/20,5/4/20,5/5/20,5/6/20,5/7/20,5/8/20,5/9/20,5/10/20,5/11/20,5/12/20,5/13/20,5/14/20,5/15/20,5/16/20,5/17/20,5/18/20,5/19/20,5/20/20,5/21/20,5/22/20,5/23/20,5/24/20,5/25/20,5/26/20,5/27/20,5/28/20,5/29/20,5/30/20,5/31/20,6/1/20,6/2/20,6/3/20,6/4/20,6/5/20,6/6/20,6/7/20,6/8/20,6/9/20,6/10/20,6/11/20,6/12/20,6/13/20,6/14/20,6/15/20,6/16/20,6/17/20,6/18/20,6/19/20,6/20/20,6/21/20,6/22/20,6/23/20,6/24/20,6/25/20,6/26/20,6/27/20,6/28/20,6/29/20,6/30/20,7/1/20,7/2/20,7/3/20,7/4/20,7/5/20,7/6/20,7/7/20,7/8/20,7/9/20,7/10/20,7/11/20,7/12/20,7/13/20,7/14/20,7/15/20,7/16/20,7/17/20,7/18/20,7/19/20,7/20/20,7/21/20,7/22/20,7/23/20,7/24/20,7/25/20,7/26/20,7/27/20,7/28/20,7/29/20,7/30/20,7/31/20,8/1/20,8/2/20,8/3/20,8/4/20,8/5/20,8/6/20,8/7/20,8/8/20,8/9/20,8/10/20,8/11/20,8/12/20,8/13/20,8/14/20,8/15/20,8/16/20,8/17/20,8/18/20,8/19/20,8/20/20,8/21/20,8/22/20,8/23/20,8/24/20,8/25/20,8/26/20,8/27/20,8/28/20,8/29/20,8/30/20,8/31/20,9/1/20,9/2/20,9/3/20,9/4/20,9/5/20,9/6/20,9/7/20,9/8/20,9/9/20,9/10/20,9/11/20,9/12/20,9/13/20,9/14/20,9/15/20,9/16/20,9/17/20,9/18/20,9/19/20,9/20/20,9/21/20,9/22/20,9/23/20,9/24/20,9/25/20,9/26/20,9/27/20,9/28/20,9/29/20,9/30/20,10/1/20,10/2/20,10/3/20,10/4/20,10/5/20,10/6/20,10/7/20,10/8/20,10/9/20,10/10/20,10/11/20,10/12/20,10/13/20,10/14/20,10/15/20,10/16/20,10/17/20,10/18/20,10/19/20,10/20/20,10/21/20,10/22/20,10/23/20,10/24/20,10/25/20,10/26/20,10/27/20,10/28/20,10/29/20,10/30/20,10/31/20,11/1/20,11/2/20,11/3/20,11/4/20,11/5/20,11/6/20,11/7/20,11/8/20,11/9/20,11/10/20,11/11/20,11/12/20,11/13/20,11/14/20,11/15/20,11/16/20,11/17/20,11/18/20,11/19/20,11/20/20,11/21/20,11/22/20,11/23/20,11/24/20,11/25/20,11/26/20,11/27/20,11/28/20,11/29/20,11/30/20,12/1/20,12/2/20,12/3/20,12/4/20,12/5/20,12/6/20,12/7/20,12/8/20,12/9/20,12/10/20,12/11/20,12/12/20,12/13/20,12/14/20,12/15/20,12/16/20,12/17/20,12/18/20,12/19/20,12/20/20,12/21/20,12/22/20,12/23/20,12/24/20,12/25/20,12/26/20,12/27/20,12/28/20,12/29/20,12/30/20,12/31/20,1/1/21,1/2/21,1/3/21,1/4/21,1/5/21,1/6/21,1/7/21,1/8/21,1/9/21,1/10/21,1/11/21,1/12/21,1/13/21,1/14/21,1/15/21,1/16/21,1/17/21,1/18/21,1/19/21,1/20/21,1/21/21,1/22/21,1/23/21,1/24/21,1/25/21,1/26/21,1/27/21,1/28/21,1/29/21,1/30/21,1/31/21,2/1/21,2/2/21,2/3/21,2/4/21,2/5/21,2/6/21,2/7/21,2/8/21,2/9/21,2/10/21,2/11/21,2/12/21,2/13/21,2/14/21,2/15/21,2/16/21,2/17/21,2/18/21,2/19/21,2/20/21,2/21/21,2/22/21,2/23/21,2/24/21,2/25/21,2/26/21,2/27/21,2/28/21,3/1/21,3/2/21,3/3/21,3/4/21,3/5/21,3/6/21,3/7/21,3/8/21,3/9/21,3/10/21,3/11/21,3/12/21,3/13/21,3/14/21,3/15/21,3/16/21,3/17/21,3/18/21,3/19/21,3/20/21,3/21/21,3/22/21,3/23/21,3/24/21,3/25/21,3/26/21,3/27/21,3/28/21,3/29/21,3/30/21,3/31/21,4/1/21,4/2/21,4/3/21,4/4/21,4/5/21,4/6/21,4/7/21,4/8/21,4/9/21,4/10/21,4/11/21,4/12/21,4/13/21,4/14/21,4/15/21,4/16/21,4/17/21,4/18/21,4/19/21,4/20/21,4/21/21,4/22/21,4/23/21,4/24/21,4/25/21,4/26/21,4/27/21,4/28/21,4/29/21,4/30/21,5/1/21,5/2/21,5/3/21,5/4/21,5/5/21,5/6/21,5/7/21,5/8/21,5/9/21,5/10/21,5/11/21,5/12/21,5/13/21,5/14/21,5/15/21,5/16/21,5/17/21,5/18/21,5/19/21,5/20/21,5/21/21,5/22/21,5/23/21,5/24/21,5/25/21,5/26/21,5/27/21,5/28/21,5/29/21,5/30/21,5/31/21,6/1/21,6/2/21,6/3/21,6/4/21,6/5/21,6/6/21,6/

In [6]:
confirmed_df.tail()

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,1/28/20,1/29/20,1/30/20,1/31/20,2/1/20,2/2/20,2/3/20,2/4/20,2/5/20,2/6/20,2/7/20,2/8/20,2/9/20,2/10/20,2/11/20,2/12/20,2/13/20,2/14/20,2/15/20,2/16/20,2/17/20,2/18/20,2/19/20,2/20/20,2/21/20,2/22/20,2/23/20,2/24/20,2/25/20,2/26/20,2/27/20,2/28/20,2/29/20,3/1/20,3/2/20,3/3/20,3/4/20,3/5/20,3/6/20,3/7/20,3/8/20,3/9/20,3/10/20,3/11/20,3/12/20,3/13/20,3/14/20,3/15/20,3/16/20,3/17/20,3/18/20,3/19/20,3/20/20,3/21/20,3/22/20,3/23/20,3/24/20,3/25/20,3/26/20,3/27/20,3/28/20,3/29/20,3/30/20,3/31/20,4/1/20,4/2/20,4/3/20,4/4/20,4/5/20,4/6/20,4/7/20,4/8/20,4/9/20,4/10/20,4/11/20,4/12/20,4/13/20,4/14/20,4/15/20,4/16/20,4/17/20,4/18/20,4/19/20,4/20/20,4/21/20,4/22/20,4/23/20,4/24/20,4/25/20,4/26/20,4/27/20,4/28/20,4/29/20,4/30/20,5/1/20,5/2/20,5/3/20,5/4/20,5/5/20,5/6/20,5/7/20,5/8/20,5/9/20,5/10/20,5/11/20,5/12/20,5/13/20,5/14/20,5/15/20,5/16/20,5/17/20,5/18/20,5/19/20,5/20/20,5/21/20,5/22/20,5/23/20,5/24/20,5/25/20,5/26/20,5/27/20,5/28/20,5/29/20,5/30/20,5/31/20,6/1/20,6/2/20,6/3/20,6/4/20,6/5/20,6/6/20,6/7/20,6/8/20,6/9/20,6/10/20,6/11/20,6/12/20,6/13/20,6/14/20,6/15/20,6/16/20,6/17/20,6/18/20,6/19/20,6/20/20,6/21/20,6/22/20,6/23/20,6/24/20,6/25/20,6/26/20,6/27/20,6/28/20,6/29/20,6/30/20,7/1/20,7/2/20,7/3/20,7/4/20,7/5/20,7/6/20,7/7/20,7/8/20,7/9/20,7/10/20,7/11/20,7/12/20,7/13/20,7/14/20,7/15/20,7/16/20,7/17/20,7/18/20,7/19/20,7/20/20,7/21/20,7/22/20,7/23/20,7/24/20,7/25/20,7/26/20,7/27/20,7/28/20,7/29/20,7/30/20,7/31/20,8/1/20,8/2/20,8/3/20,8/4/20,8/5/20,8/6/20,8/7/20,8/8/20,8/9/20,8/10/20,8/11/20,8/12/20,8/13/20,8/14/20,8/15/20,8/16/20,8/17/20,8/18/20,8/19/20,8/20/20,8/21/20,8/22/20,8/23/20,8/24/20,8/25/20,8/26/20,8/27/20,8/28/20,8/29/20,8/30/20,8/31/20,9/1/20,9/2/20,9/3/20,9/4/20,9/5/20,9/6/20,9/7/20,9/8/20,9/9/20,9/10/20,9/11/20,9/12/20,9/13/20,9/14/20,9/15/20,9/16/20,9/17/20,9/18/20,9/19/20,9/20/20,9/21/20,9/22/20,9/23/20,9/24/20,9/25/20,9/26/20,9/27/20,9/28/20,9/29/20,9/30/20,10/1/20,10/2/20,10/3/20,10/4/20,10/5/20,10/6/20,10/7/20,10/8/20,10/9/20,10/10/20,10/11/20,10/12/20,10/13/20,10/14/20,10/15/20,10/16/20,10/17/20,10/18/20,10/19/20,10/20/20,10/21/20,10/22/20,10/23/20,10/24/20,10/25/20,10/26/20,10/27/20,10/28/20,10/29/20,10/30/20,10/31/20,11/1/20,11/2/20,11/3/20,11/4/20,11/5/20,11/6/20,11/7/20,11/8/20,11/9/20,11/10/20,11/11/20,11/12/20,11/13/20,11/14/20,11/15/20,11/16/20,11/17/20,11/18/20,11/19/20,11/20/20,11/21/20,11/22/20,11/23/20,11/24/20,11/25/20,11/26/20,11/27/20,11/28/20,11/29/20,11/30/20,12/1/20,12/2/20,12/3/20,12/4/20,12/5/20,12/6/20,12/7/20,12/8/20,12/9/20,12/10/20,12/11/20,12/12/20,12/13/20,12/14/20,12/15/20,12/16/20,12/17/20,12/18/20,12/19/20,12/20/20,12/21/20,12/22/20,12/23/20,12/24/20,12/25/20,12/26/20,12/27/20,12/28/20,12/29/20,12/30/20,12/31/20,1/1/21,1/2/21,1/3/21,1/4/21,1/5/21,1/6/21,1/7/21,1/8/21,1/9/21,1/10/21,1/11/21,1/12/21,1/13/21,1/14/21,1/15/21,1/16/21,1/17/21,1/18/21,1/19/21,1/20/21,1/21/21,1/22/21,1/23/21,1/24/21,1/25/21,1/26/21,1/27/21,1/28/21,1/29/21,1/30/21,1/31/21,2/1/21,2/2/21,2/3/21,2/4/21,2/5/21,2/6/21,2/7/21,2/8/21,2/9/21,2/10/21,2/11/21,2/12/21,2/13/21,2/14/21,2/15/21,2/16/21,2/17/21,2/18/21,2/19/21,2/20/21,2/21/21,2/22/21,2/23/21,2/24/21,2/25/21,2/26/21,2/27/21,2/28/21,3/1/21,3/2/21,3/3/21,3/4/21,3/5/21,3/6/21,3/7/21,3/8/21,3/9/21,3/10/21,3/11/21,3/12/21,3/13/21,3/14/21,3/15/21,3/16/21,3/17/21,3/18/21,3/19/21,3/20/21,3/21/21,3/22/21,3/23/21,3/24/21,3/25/21,3/26/21,3/27/21,3/28/21,3/29/21,3/30/21,3/31/21,4/1/21,4/2/21,4/3/21,4/4/21,4/5/21,4/6/21,4/7/21,4/8/21,4/9/21,4/10/21,4/11/21,4/12/21,4/13/21,4/14/21,4/15/21,4/16/21,4/17/21,4/18/21,4/19/21,4/20/21,4/21/21,4/22/21,4/23/21,4/24/21,4/25/21,4/26/21,4/27/21,4/28/21,4/29/21,4/30/21,5/1/21,5/2/21,5/3/21,5/4/21,5/5/21,5/6/21,5/7/21,5/8/21,5/9/21,5/10/21,5/11/21,5/12/21,5/13/21,5/14/21,5/15/21,5/16/21,5/17/21,5/18/21,5/19/21,5/20/21,5/21/21,5/22/21,5/23/21,5/24/21,5/25/21,5/26/21,5/27/21,5/28/21,5/29/21,5/30/21,5/31/21,6/1/21,6/2/21,6/3/21,6/4/21,6/5/21,6/6/21,6/

In [7]:
confirmed_df.shape

(289, 1057)

In [8]:
base_stats = pd.DataFrame(columns=['Dates','Confirmed','Deaths','Recovered','Active'])
base_stats['Dates'] = confirmed_df.columns[4:]

base_stats['Confirmed'] = base_stats['Dates'].apply(lambda x: confirmed_df[x].sum())
base_stats['Deaths'] = base_stats['Dates'].apply(lambda x: deaths_df[x].sum())
base_stats['Recovered'] = base_stats['Dates'].apply(lambda x: recoveries_df[x].sum())
base_stats.reset_index(drop=False, inplace=True)
base_stats['Active'] = base_stats['index'].apply(lambda x: (base_stats['Confirmed'][x]-(base_stats['Deaths'][x]+base_stats['Recovered'][x])))
base_stats.head()

,index,Dates,Confirmed,Deaths,Recovered,Active
0,0,1/22/20,557,17,30,510
1,1,1/23/20,657,18,32,607
2,2,1/24/20,944,26,39,879
3,3,1/25/20,1437,42,42,1353
4,4,1/26/20,2120,56,56,2008


** 4. Explatory Data Analysis of Latest Statistics for COVID-19 **

In [9]:
latest_stats_fig = go.Figure()
latest_stats_fig.add_trace(go.Treemap(labels = ['Confirmed','Active','Recovered','Deaths'],
                                     parents = ['','Confirmed','Confirmed','Confirmed'],
                                     values = [base_stats['Confirmed'].sum(), base_stats['Active'].sum(), base_stats['Recovered'].sum(), base_stats['Deaths'].sum()],
                                      branchvalues="total", marker_colors = ['#118ab2','#ef476f','#06d6a0','#073b4c'],
                                      textinfo = "label+text+value",
                                      outsidetextfont = {"size": 30, "color": "darkblue"},
                                      marker = {"line": {"width": 2}},
                                        pathbar = {"visible": False}
                                     ))
latest_stats_fig.update_layout(#width=1000, 
                               height=300)
latest_stats_fig.show()

**Function for Basic Statistics of COVID-19**

In [10]:
base_stats_fig = go.Figure()

for column in base_stats.columns.to_list()[2:6]:
    color_dict = {
      "Confirmed": "#118ab2",
      "Active": "#ef476f",
      "Recovered": "#06d6a0",
      "Deaths": "#073b4c"
        }
    base_stats_fig.add_trace(
        go.Scatter(
            x = base_stats['Dates'],
            y = base_stats[column],
            name = column,
            line = dict(color=color_dict[column]),
            hovertemplate ='<br><b>Date</b>: %{x}'+'<br><i>Count</i>:'+'%{y}',
        )
    )

    for column in base_stats.columns.to_list()[2:6]:
        color_dict = {
      "Confirmed": "#149ECC",
      "Active": "#F47C98",
      "Recovered": "#24F9C1",
      "Deaths": "#0C6583"
        }
    base_stats_fig.add_trace(
        go.Scatter(
            x = base_stats['Dates'],
            y = base_stats['index'].apply(lambda x: (base_stats[column][x-7:x].sum())/7 if x>7 else (base_stats[column][0:x].sum())/7),
            name = column+" 7-day Moving Avg.",
            line = dict(dash="dash", color=color_dict[column]), showlegend=False,
            hovertemplate = '<br><b>Date</b>: %{x}'+'<br><i>7-day moving avg.</i>: %{y}'
        )
    )

    base_stats_fig.update_layout(
    updatemenus=[
        dict(
        buttons=list(
            [dict(label = 'All Cases',
                  method = 'update',
                  args = [{'visible': [True, True, True, True, True, True, True, True]},
                          {'title': 'All Cases',
                           'showlegend':True}]),
             dict(label = 'Confirmed',
                  method = 'update',
                  args = [{'visible': [True, False, False, False, True, False, False, False]},
                          {'title': 'Confirmed',
                           'showlegend':True}]),
             dict(label = 'Active',
                  method = 'update',
                  args = [{'visible': [False, False, False, True, False, False, False, True]},

                           {'title': 'Active',
                           'showlegend':True}]),
             dict(label = 'Recovered',
                  method = 'update',
                  args = [{'visible': [False, False, True, False, False, False, True, False]},
                          {'title': 'Recovered',
                           'showlegend':True}]),
             dict(label = 'Deaths',
                  method = 'update',
                  args = [{'visible': [False, True, False, False, False, True, False, False]},
                          {'title': 'Deaths',
                           'showlegend':True}]),
            ]),
             type = "dropdown",
             direction="down",
#             pad={"r": 10, "t": 40},
             showactive=True,
             x=0,
             xanchor="left",
             y=1.25,
             yanchor="top"
        ),
        dict(
        buttons=list(
            [dict(label = 'Linear Scale',
                  method = 'relayout',
                  args = [{'yaxis': {'type': 'linear'}},
                          {'title': 'All Cases',
                           'showlegend':True}]),
             dict(label = 'Log Scale',
                  method = 'relayout',
                  args = [{'yaxis': {'type': 'log'}},
                          {'title': 'Confirmed',
                           'showlegend':True}]),
            ]),
             type = "dropdown",
             direction="down",
#             pad={"r": 10, "t": 10},
             showactive=True,
             x=0,
             xanchor="left",
             y=1.36,
             yanchor="top"
        )
    ])

    base_stats_fig.update_xaxes(showticklabels=False)
base_stats_fig.update_layout(
    #height=600, width=600, 
    title_text="Basic Statistics for Covid19", title_x=0.5, title_font_size=20,
                            legend=dict(orientation='h',yanchor='top',y=1.15,xanchor='right',x=1), paper_bgcolor="mintcream",
                            xaxis_title="Date", yaxis_title="# of Cases")
base_stats_fig.show()


**Function For Finding Important Ratios**

In [11]:
imp_ratios_fig = go.Figure()
color_dict = {
  "Confirmed": ["#118ab2",(base_stats['Deaths']/base_stats['Confirmed'])*100, 'Deaths/100 Cases','#149ECC'],
  "Active": ["#ef476f",(base_stats['Deaths']/base_stats['Recovered'])*100, 'Deaths/100 Recovered','#F47C98'],
  "Recovered": ["#06d6a0",(base_stats['Recovered']/base_stats['Confirmed'])*100, 'Recovered/100 cases','#24F9C1'],
  "Deaths": ["#073b4c",(base_stats['Recovered']/base_stats['Deaths'])*100, 'Recovered/100 Deaths','#0C6583']
    }
for column in base_stats.columns.to_list()[2:6]:

    imp_ratios_fig.add_trace(go.Scatter(x = base_stats['Dates'],y = color_dict[column][1],
                                        name = color_dict[column][2],line = dict(color=color_dict[column][0]),
            hovertemplate ='<br><b>Date</b>: %{x}'+'<br><i>Ratio </i>:'+'%{y}'))
    imp_ratios_fig.add_trace(go.Scatter(x = base_stats['Dates'],y = [color_dict[column][1].mean()]*base_stats['Dates'].shape[0],
            name = "Mean value",line = dict(dash="dash", color=color_dict[column][3]),
            hovertemplate ='<br><i>Mean value </i>:'+'%{y}', visible=False))

imp_ratios_fig.update_layout(
    updatemenus=[
        dict(
        buttons=list(
            [dict(label = 'All Ratios',
                  method = 'update',
                  args = [{'visible': [True, False, True, False, True, False, True, False]},
                          {'title': 'All Cases',
                           'showlegend':True}]),
             dict(label = 'Deaths/100 Cases<br>(Mortality rate)',
                  method = 'update',
                  args = [{'visible': [True, True, False, False, False, False, False, False]},
                          {'title': 'Confirmed',
                           'showlegend':True}]),
             dict(label = 'Deaths/100 Recovered',
                  method = 'update',
                  args = [{'visible': [False, False, True, True, False, False, False, False]},
                          {'title': 'Active',
                           'showlegend':True}]),
             dict(label = 'Recovered/100 cases<br>(Recovery rate)',
                  method = 'update',
                  args = [{'visible': [False, False, False, False, True, True, False, False]},
                          {'title': 'Recovered',
                           'showlegend':True}]),
             dict(label = 'Recovered/100 Deaths',
                  method = 'update',
                  args = [{'visible': [False, False, False, False, False, False, True, True]},
                          {'title': 'Deaths',
                           'showlegend':True}]),
            ]),
             type = "buttons",
             direction="down",
#             pad={"r": 10, "t": 40},
             showactive=True,
             x=1.01,
             xanchor="left",
             y=1,
             yanchor="top"
        )
    ])

In [12]:
imp_ratios_fig.update_xaxes(showticklabels=False)
imp_ratios_fig.update_layout(
    #height=500, width=1100, 
    title_text="Important Ratios for Covid19", title_x=0.5, title_font_size=20,
                            legend=dict(orientation='h',yanchor='top',y=1.12,xanchor='right',x=1), paper_bgcolor="mintcream",
                            xaxis_title="Date", yaxis_title="Ratio")
imp_ratios_fig.show()

** 5. Prediction on Statistics of COVID 19 cases**

In [13]:
base_stats_inc_df = pd.DataFrame(columns=['Index', 'Dates', 'Confirmed', 'Deaths', 'Recovered', 'Active', 'Daily Inc.'])
base_stats_inc_df[['Index', 'Dates', 'Confirmed', 'Deaths', 'Recovered', 'Active']] = base_stats[['index', 'Dates', 'Confirmed', 'Deaths', 'Recovered', 'Active']]
base_stats_inc_df['Daily Inc.'] = base_stats['index'].apply(lambda x: base_stats['Confirmed'][x]-base_stats['Confirmed'][x-1:x].sum())

days = np.array(base_stats_inc_df[['Index']]).reshape(-1, 1)
days_ex = []
for i in range(len(days)+30):
    days_ex = days_ex+[[i]]
prediction_df = pd.DataFrame(columns=['Index', 'Confirmed Pred', 'Deaths Pred', 'Recovered Pred', 'Active Pred', 'Daily Inc. Pred'])
prediction_df['Index'] = list(flatten(days_ex))

for col in base_stats_inc_df.columns[2:]:

    count = np.array(base_stats_inc_df[[col]]).reshape(-1, 1)

    X_train_confirmed, X_test_confirmed, y_train_confirmed, y_test_confirmed = train_test_split(
                                                        days[50:], count[50:], 
                                                        test_size=0.05, shuffle=False)

    MAE, RSE, R2 = [], [], []
    for j in range(1,10):
        #creating the model
        poly = PolynomialFeatures(degree=j)
        train_x_poly = poly.fit_transform(X_train_confirmed)
        regr_poly = linear_model.LinearRegression()
        regr_poly.fit(train_x_poly, y_train_confirmed)
        
        y_pred_poly = regr_poly.predict(poly.fit_transform(X_test_confirmed))
        MAE.append(np.mean(np.absolute(y_pred_poly - y_test_confirmed)))
        RSE.append(np.mean((y_pred_poly - list(flatten(y_test_confirmed))) ** 2))
        R2.append(r2_score(y_pred_poly, list(flatten(y_test_confirmed))))
        
    deg = RSE.index(min(RSE))+1
    #print("best deg for column {} is {}".format(col, deg))

    poly = PolynomialFeatures(degree=deg)
    train_x_poly = poly.fit_transform(X_train_confirmed)

    regr_poly = linear_model.LinearRegression()
    regr_poly.fit(train_x_poly, y_train_confirmed)
    col_name = col+' Pred'
    prediction_df[col_name] = list(flatten(regr_poly.predict(poly.fit_transform(days_ex))))
prediction_fig = go.Figure()
pred_dict = {
  "Confirmed": ["#118ab2", 'Confirmed', 'Predicted Confirmed','#149ECC'],
  "Active": ["#ef476f", 'Deaths', 'Predicted Deaths', '#F47C98'],
  "Recovered": ["#06d6a0", 'Recovered', 'Predicted Recovered','#24F9C1'],
  "Deaths": ["#073b4c", 'Active', 'Predicted Active','#0C6583'],
  "Daily Inc.": ["black", 'Daily Inc.', 'Predicted Daily Inc.','grey']
    }
for z in base_stats_inc_df.columns[2:]:
    
    z_pred = z+' Pred'
    prediction_fig.add_trace(go.Scatter(x=list(flatten(days)), y=base_stats_inc_df[z],
                                       line=dict(color=pred_dict[z][0]), name = pred_dict[z][1],
                                       hovertemplate ='<br><b>Day number</b>: %{x}'+'<br><i>No.of cases </i>:'+'%{y}'))
    
    prediction_fig.add_trace(go.Scatter(x=list(flatten(days_ex))[50:], y=prediction_df[z_pred][50:],
                                       line=dict(dash="dash", color='black'), visible=False, name = pred_dict[z][2],
                                       hovertemplate ='<br><b>Day number</b>: %{x}'+'<br><i>Predicted no.of cases </i>:'+'%{y}'))
    
    prediction_fig.update_layout(
    updatemenus=[
        dict(
        buttons=list(
            [dict(label = 'Confirmed',
                  method = 'update',
                  args = [{'visible': [True, True, False, False, False, False, False, False, False, False]},
                          {'title': 'Confirmed Cases',
                           'showlegend':True}]),
             dict(label = 'Deaths',
                  method = 'update',
                  args = [{'visible': [False, False, True, True, False, False, False, False, False, False]},
                          {'title': 'Deaths Cases',
                           'showlegend':True}]),
             dict(label = 'Recovered',
                  method = 'update',
                  args = [{'visible': [False, False, False, False, True, True, False, False, False, False]},
                          {'title': 'Recovered Cases',
                           'showlegend':True}]),
             dict(label = 'Active',
                  method = 'update',
                  args = [{'visible': [False, False, False, False, False, False, True, True, False, False]},
                          {'title': 'Active Cases',
                           'showlegend':True}]),
             dict(label = 'Daily Inc.',
                  method = 'update',
                  args = [{'visible': [False, False, False, False, False, False, False, False, True, True]},
                          {'title': 'Daily Inc. Cases',
                           'showlegend':True}]),
            ]),
             type = "buttons",
             direction="down",
             # pad={"r": 10, "t": 40},
             showactive=True,
             # x=1.01,
             # xanchor="left",
             y=1.1,
             yanchor="top"
        )
    ])


prediction_fig.update_xaxes(showticklabels=False)
prediction_fig.update_layout(
    #height=500, width=1100, 
    title_text="Prediction for Covid19 Cases", title_x=0.5, title_font_size=20,
    legend=dict(orientation='h',yanchor='top',y=1.12,xanchor='right',x=1), paper_bgcolor="mintcream",
                            xaxis_title="Number of Days <br> (Click on the buttons at the left to see the predictions)", yaxis_title="Count")
prediction_fig.show()


**Co-relation Matrix**

In [14]:
corr_mat_fig = go.Figure()

corr_mat_fig = go.Figure(data=go.Splom(
                dimensions=[dict(label='Confirmed',
                                 values=base_stats['Confirmed']),
                            dict(label='Active',
                                 values=base_stats['Active']),
                            dict(label='Recovered',
                                 values=base_stats['Recovered']),
                            dict(label='Deaths',
                                 values=base_stats['Deaths'])],
                text=base_stats['Dates'],
    diagonal_visible=False,
        marker=dict(color='red',
                showscale=False, # colors encode categorical variables
                line_color='white', line_width=0.5)
                ))


corr_mat_fig.update_layout(
    #height=600, width=600, 
                                     title_text="Correlation Matrix for types of Cases",
                                     title_x=0.5, title_font_size=15, paper_bgcolor="mintcream",
                                     legend=dict(orientation='h',yanchor='top',y=1.12,xanchor='right',x=1))

corr_mat_fig.show()

**HeatMap Co-relation Between Cases**

In [15]:
corr = []
for i in base_stats.columns[2:]:
    temp = []
    for j in base_stats.columns[2:]:
        temp.append(base_stats[j].sum()/base_stats[i].sum())
    corr.append(temp)

ff_heatmap = go.Figure(data=go.Heatmap(
        z=corr,
        x=base_stats.columns[2:],
        y=base_stats.columns[2:],
        colorscale='reds'))

ff_heatmap.update_layout(title_text='title', title_x=0.5, 
                   width=600, height=600,
                   xaxis_showgrid=False,
                   yaxis_showgrid=False,
                   yaxis_autorange='reversed'
                                           )
ff_heatmap.update_layout(title_text="HeatMap showing Corr. between Cases",
                                     title_x=0.5, title_font_size=20,
                                     paper_bgcolor="mintcream")#height=300, width=1100,
ff_heatmap.show()